In [1]:
import folium
import pandas as pd
import pickle
import numpy as np

In [2]:
file = open('./pickles/house_latlong.p','rb')
df = pickle.load(file)

In [10]:
ave_lat = np.mean(df.latitude)
ave_long = np.mean(df.longitude)
(ave_lat, ave_long)

(42.03438790059312, -93.64255447330521)

In [70]:
neighborhoods = df[~np.isnan(df.latitude)].groupby('Neighborhood')[['latitude','longitude']].mean() #get center of neighborhood
neighborhoods['num_houses'] = df.Neighborhood.value_counts()
nieghborhoods = neighborhoods.reset_index()

In [71]:
m = folium.Map(location=[ave_lat, ave_long],tiles='Stamen Toner')
for i in range(0,neighborhoods.shape[0]):
    folium.Circle(location = [neighborhoods.latitude[i], neighborhoods.longitude[i]], 
                  radius = int(neighborhoods.num_houses[i]), popup = nieghborhoods.Neighborhood[i]).add_to(m)
m    
    #color the circles

In [4]:
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.model_selection import train_test_split

In [268]:
KNR = KNeighborsRegressor(n_neighbors=50)

In [8]:
#dropping NA values with lat long. 
df = df[~np.isnan(df.latitude)]
#redo this with the cosine(42) multiplication. 
df['e_long'] = df['longitude']*np.cos(np.radians(42))

In [9]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('SalePrice', axis = 1),df['SalePrice'], test_size = .5)

In [326]:
KNR.fit(x_train[['latitude','e_long']],y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=2,
                    weights='uniform')

In [321]:
from folium.plugins import HeatMap
m = folium.Map(location=[ave_lat, ave_long],tiles='Stamen Toner')
x_train['predict'] = KNR.predict(x_train[['latitude','e_long']])/50000
HeatMap(x_train[['latitude','longitude','predict']].values).add_to(m)
m

In [327]:
len(np.unique(KNR.predict(x_train[['latitude','e_long']])))

873

In [329]:
KNR.score(x_train[['latitude','e_long']],y_train), KNR.score(x_test[['latitude','e_long']],y_test) #obviously can make this better. Do a grid search, just have to

(0.5661020285780276, 0.5344456371666844)

In [330]:
x_train['residuals'] = y_train - KNR.predict(x_train[['latitude','e_long']])

In [331]:
x_train.residuals

1407    34272.38
2117    14522.50
1808     4894.74
1150   -16654.08
1664    -4493.00
          ...   
741    -52240.60
1699     1805.00
1153   -71858.44
1494     7938.00
273    -21551.30
Name: residuals, Length: 1268, dtype: float64

### SVR - with RBF Kernels

In [6]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [17]:
SVR_map = make_pipeline(StandardScaler(),SVR(epsilon =.0001 ,C = 100000)) #NEED TO USE GRIDSEARCH!!! Maybe fit toooo
SVR_map.fit(x_train[['latitude','e_long']],y_train)
SVR_map.score(x_train[['latitude','e_long']], y_train), SVR_map.score(x_test[['latitude','e_long']],y_test)

(0.4508520961413245, 0.4496386000911568)

In [18]:
x_train['residuals'] = y_train - SVR_map.predict(x_train[['latitude','e_long']])

### Random Forest

In [19]:
from sklearn.linear_model import LinearRegression

house_base_reduced = x_train.loc[:,['GrLivArea','residuals','LotArea',
                      'OverallQual','OverallCond','YearBuilt','YearRemodAdd',
                      'TotalBsmtSF','Fireplaces','GarageArea',
                      'MoSold','YrSold']]
house_base_reduced = pd.get_dummies(house_base_reduced)

house_base_reduced['Age'] = house_base_reduced['YrSold'] - house_base_reduced['YearBuilt'] #think about combining into one
house_base_reduced['YrSinceRemod'] =  house_base_reduced['YrSold'] - house_base_reduced['YearRemodAdd']
house_base_reduced = house_base_reduced.drop(['YrSold','YearBuilt','YearRemodAdd'],axis = 1)

In [20]:
lm_residuals = LinearRegression()

In [21]:
house_base_reduced = house_base_reduced.dropna()

In [22]:
lm_residuals.fit(house_base_reduced.drop('residuals', axis =1), house_base_reduced['residuals'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
import statsmodels.api as sm
x = sm.add_constant(house_base_reduced.drop('residuals', axis =1))
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(house_base_reduced['residuals'], x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              residuals   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     159.4
Date:                Wed, 26 Aug 2020   Prob (F-statistic):          1.93e-215
Time:                        22:59:35   Log-Likelihood:                -15167.
No. Observations:                1268   AIC:                         3.036e+04
Df Residuals:                    1257   BIC:                         3.041e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -2.124e+05   1.02e+04    -20.884   

### Unsupervised Clustering

Similar analysis as above but instead use an unsupervised clustering algorithm, then just the average price in that region (think about the type of average to use). Orrrrrrrr do a separate analysis on each cluster? Idk I don't like this. 